In [1]:
import pandas as pd
import string

#nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import tokenize
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer, SentimentIntensityAnalyzer
from nltk.sentiment.util import *

#sci-kit learn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
nltk.download('stopwords')
stop = stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wordnet= WordNetLemmatizer()
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

In [3]:
def lower(text):
    return text.lower()

In [4]:
def tokenization(text):
    tokens =  nltk.word_tokenize(text)
    return tokens

In [5]:
def remove_stopWord(text):
    output=[i for i in text if i not in stop]
    return output

In [6]:
def lemmatizer(words):
    return [wordnet.lemmatize(word) for word in words]
    

In [7]:
def preprocess(text):
    text1=remove_punctuation(text)
    text2=lower(text1)
    word=tokenization(text2)
    words=remove_stopWord(word)
    final=lemmatizer(words)
    return final

In [8]:
# final_word = preprocess(text)
# print(final_word)

In [9]:
data=pd.read_csv('IMDB Dataset.csv')

In [10]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [11]:
data['review']=data['review'].apply(lambda x:x.lower())

In [12]:
data["review"]=data["review"].apply(lambda x: preprocess(x))

In [13]:
data.head()

,review,sentiment
0,"[one, reviewer, mentioned, watching, 1, oz, ep...",positive
1,"[wonderful, little, production, br, br, filmin...",positive
2,"[thought, wonderful, way, spend, time, hot, su...",positive
3,"[basically, there, family, little, boy, jake, ...",negative
4,"[petter, matteis, love, time, money, visually,...",positive


In [14]:
import inflect
def replace_numbers(text):
    p = inflect.engine()
    new_words = []
    for word in text:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

In [15]:
data["review"]=data["review"].apply(lambda x:replace_numbers(x))

In [16]:
data.head()

,review,sentiment
0,"[one, reviewer, mentioned, watching, one, oz, ...",positive
1,"[wonderful, little, production, br, br, filmin...",positive
2,"[thought, wonderful, way, spend, time, hot, su...",positive
3,"[basically, there, family, little, boy, jake, ...",negative
4,"[petter, matteis, love, time, money, visually,...",positive


In [17]:
data["review"]=data["review"].apply(lambda x:replace_numbers(x))

In [18]:
data['lemmatized'] = data.review.apply(lambda x: ' '.join(x))

In [19]:
data.head()

,review,sentiment,lemmatized
0,"[one, reviewer, mentioned, watching, one, oz, ...",positive,one reviewer mentioned watching one oz episode...
1,"[wonderful, little, production, br, br, filmin...",positive,wonderful little production br br filming tech...
2,"[thought, wonderful, way, spend, time, hot, su...",positive,thought wonderful way spend time hot summer we...
3,"[basically, there, family, little, boy, jake, ...",negative,basically there family little boy jake think t...
4,"[petter, matteis, love, time, money, visually,...",positive,petter matteis love time money visually stunni...


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [21]:
tfidf=TfidfVectorizer(max_features=10000)

In [22]:
x=data["lemmatized"]
y=data["sentiment"]

In [23]:
x

0        one reviewer mentioned watching one oz episode...
1        wonderful little production br br filming tech...
2        thought wonderful way spend time hot summer we...
3        basically there family little boy jake think t...
4        petter matteis love time money visually stunni...
                               ...                        
49995    thought movie right good job wasnt creative or...
49996    bad plot bad dialogue bad acting idiotic direc...
49997    catholic taught parochial elementary school nu...
49998    im going disagree previous comment side maltin...
49999    one expects star trek movie high art fan expec...
Name: lemmatized, Length: 50000, dtype: object

In [24]:
x=tfidf.fit_transform(x)

In [25]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [26]:
clf=LinearSVC()

In [27]:
clf.fit(x_train,y_train)

LinearSVC()

In [28]:
y_pred=clf.predict(x_test)

In [29]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

    negative       0.89      0.88      0.88      5035
    positive       0.88      0.89      0.88      4965

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [30]:
tf="this movie is really good."
tf=preprocess(tf)
print(tf)

['movie', 'really', 'good']


In [31]:
tf= " ".join(tf)
tf

'movie really good'

In [32]:
vc=tfidf.transform([tf])

In [33]:
vc.shape

(1, 10000)

In [34]:
clf.predict(vc)

array(['positive'], dtype=object)

In [35]:
tf="this movie is really bad."
tf=preprocess(tf)
print(tf)

['movie', 'really', 'bad']


In [36]:
tf= " ".join(tf)
tf

'movie really bad'

In [37]:
vc=tfidf.transform([tf])

In [38]:
vc.shape

(1, 10000)

In [39]:
clf.predict(vc)

array(['negative'], dtype=object)

In [40]:
import pickle
pickle.dump(clf,open('IMDB_model','wb'))